<a href="https://colab.research.google.com/github/liuwang2000/colabScript/blob/main/notebooks/predict_TM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TM prediction with Prime

This tutorial demonstrates how to predict the OGT of a protein using a pretrained model from the Prime model.

We provide:

- The sequences, a FASTA file.

Goals
Obtain an predicted TM for each sequence.


## Import the necessary libraries and modules.

In [2]:
# 克隆仓库（如果尚未克隆）
!git clone https://github.com/ai4protein/Pro-Prime.git
%cd Pro-Prime/notebooks
# 步骤1：挂载Google Drive（每次重启后需重新挂载）
from google.colab import drive
drive.mount('/content/drive')

# 步骤2：安装依赖
!pip install torch==2.4
!pip install torch_xla==2.4
!pip install biopython==1.81

Cloning into 'Pro-Prime'...
remote: Enumerating objects: 1244, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 1244 (delta 43), reused 36 (delta 16), pack-reused 1173 (from 1)
Receiving objects: 100% (1244/1244), 67.98 MiB | 31.48 MiB/s, done.
Resolving deltas: 100% (291/291), done.
Updating files: 100% (814/814), done.
/content/Pro-Prime/notebooks/Pro-Prime/notebooks
Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 476.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 471.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.3 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
from Bio import SeqIO
import os # Import the os modul
from tqdm.notebook import tqdm
from transformers import AutoModel

## Prepare data path

In [4]:
sequence_file = "test.fasta"

## Load model and Tokenizer

In [ ]:
!huggingface-cli login # 确保登录成功

model_path = "AI4Protein/Prime_690M"  # 再次确认模型路径
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Remove `from_flax=True` as the model might not be in Flax format
model = AutoModel.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

model.eval()
model = model.to(device)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

## Prediction

In [ ]:
tm = []
with torch.no_grad():
    for record in tqdm(list(SeqIO.parse(sequence_file, "fasta"))):
        sequence = str(record.seq)
        tokenied_results = tokenizer(sequence, return_tensors="pt")
        input_ids = tokenied_results.input_ids.to(device)
        attention_mask = tokenied_results.attention_mask.to(device)
        logits = model(input_ids=input_ids, attention_mask=attention_mask).predicted_values
        tm.append(logits.item())